` Desafio proposto: Analisar dados de comércio exterior (fonte: gov.br), referente ao período de 2020 a 2021. Principais pontos a serem abordados:
1- Quais os 3 principais produtos mais exportados por estado nos anos de 2020 e 2021.
2- Quais os 3 principais produtos mais importados por estado nos anos de 2020 e 2021.
3- Quais são os 3 principais produtos exportados em cada mês de 2021 por estado.
`

In [46]:
# importando bibliotecas
import pandas as pd # manipulação e análise
import sqlite3 #carregamento dos dados banco local
import plotly.express as px #graficos
import math

#exibir todas as colunas no df
pd.set_option("display.max_columns", None)

In [47]:
# carregamento
from google.colab import drive
drive.mount('/content/drive')

# acesso a pasta com os arquivos
pasta = "/content/drive/MyDrive/dados_exportacao/"

# arquivos:
exp_2020 = pd.read_csv(pasta + "EXP_2020.csv", sep=";", encoding="latin1")
exp_2021 = pd.read_csv(pasta + "EXP_2021.csv", sep=";", encoding="latin1")
imp_2020 = pd.read_csv(pasta + "IMP_2020.csv", sep=";", encoding="latin1")
imp_2021 = pd.read_csv(pasta + "IMP_2021.csv", sep=";", encoding="latin1")


# concatenação dos arquivos
df_exp_total = pd.concat([exp_2020, exp_2021], ignore_index=True)
df_imp_total = pd.concat([imp_2020, imp_2021], ignore_index=True)

# limpeza adicional: identifiquei no csv casos em que existiam tuplas das colunas referentes a UF com dados "EX" e "ND", como não é solicitado esse tipo de informação para as analises em questão, estou filtrando.
df_exp_total = df_exp_total[~df_exp_total["SG_UF_NCM"].isin(["ND", "EX"])]
df_imp_total = df_imp_total[~df_imp_total["SG_UF_NCM"].isin(["ND", "EX"])]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
# etapa de verificação de qualidade dos dados importados
print("ETAPA DE VERIFICAÇÃO DA QUALIDADE DE DADOS IMPORTADOS REFERENTE A EXPORTAÇÃO")
print("Número de linhas:", len(df_exp_total))
print("Valores nulos por coluna:\n", df_exp_total.isnull().sum())
print("Duplicados:", df_exp_total.duplicated().sum())
print("Anos distintos:", df_exp_total["CO_ANO"].unique())
print("Estados distintos:", df_exp_total["SG_UF_NCM"].unique())

print("ETAPA DE VERIFICAÇÃO DA QUALIDADE DE DADOS IMPORTADOS REFERENTE A IMPORTAÇÃO")
print("Número de linhas:", len(df_imp_total))
print("Valores nulos por coluna:\n", df_imp_total.isnull().sum())
print("Duplicados:", df_imp_total.duplicated().sum())
print("Anos distintos:", df_imp_total["CO_ANO"].unique())
print("Estados distintos:", df_imp_total["SG_UF_NCM"].unique())

ETAPA DE VERIFICAÇÃO DA QUALIDADE DE DADOS IMPORTADOS REFERENTE A EXPORTAÇÃO
Número de linhas: 2821080
Valores nulos por coluna:
 CO_ANO        0
CO_MES        0
CO_NCM        0
CO_UNID       0
CO_PAIS       0
SG_UF_NCM     0
CO_VIA        0
CO_URF        0
QT_ESTAT      0
KG_LIQUIDO    0
VL_FOB        0
dtype: int64
Duplicados: 0
Anos distintos: [2020 2021]
Estados distintos: ['SP' 'SC' 'CE' 'ES' 'GO' 'RS' 'AL' 'PR' 'BA' 'MG' 'RJ' 'PA' 'MT' 'MA'
 'AM' 'PE' 'MS' 'RR' 'TO' 'DF' 'RO' 'AP' 'AC' 'PI' 'RN' 'PB' 'SE']
ETAPA DE VERIFICAÇÃO DA QUALIDADE DE DADOS IMPORTADOS REFERENTE A IMPORTAÇÃO
Número de linhas: 3783929
Valores nulos por coluna:
 CO_ANO        0
CO_MES        0
CO_NCM        0
CO_UNID       0
CO_PAIS       0
SG_UF_NCM     0
CO_VIA        0
CO_URF        0
QT_ESTAT      0
KG_LIQUIDO    0
VL_FOB        0
VL_FRETE      0
VL_SEGURO     0
dtype: int64
Duplicados: 0
Anos distintos: [2020 2021]
Estados distintos: ['PR' 'ES' 'RS' 'SP' 'MG' 'CE' 'BA' 'PA' 'SC' 'MT' 'RJ' 'DF' 'AL' 'AM'

In [32]:
#criando conexão com o banco
conn = sqlite3.connect("comex.db")
# abre um arquivo de banco comex.db no ambiente
df_exp_total.to_sql("exportacao", conn, if_exists="replace", index=False)
df_imp_total.to_sql("importacao", conn, if_exists="replace", index=False) # salvando por tabelas separadas


3783929

In [23]:
# Top 3 principais produtos mais exportados por estado nos anos de 2020 e 2021.


# agrupar os dados de importação por UF e produto, somando a qtd total e criar o ranking
#with agregados: soma qtd p estado e produto
query_top3_exp = """
WITH agregados AS (
    SELECT SG_UF_NCM AS UF,
           CO_NCM,
           SUM(QT_ESTAT) AS QUANTIDADE_TOTAL
    FROM exportacao
    WHERE CO_ANO IN (2020, 2021)
    GROUP BY UF, CO_NCM
),
ranqueados AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY UF ORDER BY QUANTIDADE_TOTAL DESC) AS posicao
    FROM agregados
)
SELECT UF, CO_NCM, QUANTIDADE_TOTAL, posicao
FROM ranqueados
WHERE posicao <= 3
ORDER BY UF, posicao;
"""
#ROW_NUMBER () OvER (PARTITION BY UF ORDER BY QUANTIDADE_TOTAL DESC): para criar o Ranking dentro do estado
top3_exp_estado = pd.read_sql(query_top3_exp, conn)
display(top3_exp_estado)
print("Total de linhas (27 estados x 3 produtos):", len(top3_exp_estado))



,UF,CO_NCM,QUANTIDADE_TOTAL,posicao
0,AC,10059010,11771090,1
1,AC,8012100,9675258,2
2,AC,72044900,5968530,3
3,AL,22071090,27608182,1
4,AL,10059010,24818124,2
...,...,...,...,...
76,SP,20091200,2811166996,2
77,SP,71039900,2792226647,3
78,TO,10059010,1348380298,1
79,TO,2023000,141264096,2


Total de linhas (27 estados x 3 produtos): 81


In [33]:
#visualizaçao
fig_exp = px.bar(
    top3_exp_estado,
    x="UF",
    y="QUANTIDADE_TOTAL", # soma de QT_STAT é a altura
    color=top3_exp_estado["CO_NCM"].astype(str),  # transforma em string para ser categórico
    text="QUANTIDADE_TOTAL",
    title="Top 3 produtos mais exportados por estado (2020–2021)",
    labels={
        "QUANTIDADE_TOTAL": "Quantidade total",
        "CO_NCM": "Código NCM",
        "UF": "Estado"
    }
)
# ajuste no grafico
fig_exp.update_traces(texttemplate='%{text:.2s}', textposition='outside')  # reduz tamanho dos rótulos
fig_exp.update_layout(
    barmode='group', # barras lado a lado
    xaxis={'categoryorder':'category ascending'}, # oRdem UF
    legend_title_text="Código NCM"
)

fig_exp.show()

In [34]:
# Top 3 principais produtos mais importados por estado nos anos de 2020 e 2021

#mesma lógica do caso anterior
query_top3_imp = """
WITH agregados AS (
    SELECT SG_UF_NCM AS UF,
           CO_NCM,
           SUM(QT_ESTAT) AS QUANTIDADE_TOTAL
    FROM importacao
    WHERE CO_ANO IN (2020, 2021)
    GROUP BY UF, CO_NCM
),
ranqueados AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY UF ORDER BY QUANTIDADE_TOTAL DESC) AS posicao
    FROM agregados
)
SELECT UF, CO_NCM, QUANTIDADE_TOTAL, posicao
FROM ranqueados
WHERE posicao <= 3
ORDER BY UF, posicao;
"""

top3_imp_estado = pd.read_sql(query_top3_imp, conn)
display(top3_imp_estado)


,UF,CO_NCM,QUANTIDADE_TOTAL,posicao
0,AC,11010010,560000,1
1,AC,32029011,520000,2
2,AC,28332960,360000,3
3,AL,25010019,472833967,1
4,AL,29031500,381467070,2
...,...,...,...,...
76,SP,85332120,16519748246,2
77,SP,85045000,5949424280,3
78,TO,31042090,171229061,1
79,TO,27079990,150941771,2


In [45]:

# visualizaçao (importações)
fig_imp = px.bar(
    top3_imp_estado,
    x="UF",
    y="QUANTIDADE_TOTAL",
    color=top3_imp_estado["CO_NCM"].astype(str),  # força categórico
    text="QUANTIDADE_TOTAL",
    title="Top 3 produtos mais importados por estado (2020–2021)",
    labels={
        "QUANTIDADE_TOTAL": "Quantidade total",
        "CO_NCM": "Código NCM",
        "UF": "Estado"
    }  # rótulos
)

# ajuste no grafico
fig_imp.update_traces(
    texttemplate='%{text:.2s}',  # reduz tamanho dos rótulos 2 primeiros digítos
    textposition='outside'
)
fig_imp.update_layout(
    barmode='group', # barras lado a lado
    xaxis={'categoryorder': 'category ascending'}, # ordem UF
    legend_title_text="Código NCM"
)

fig_imp.show()

In [36]:
  #Top 3 produtos exportados em cada mês de 2021 por estado

# agrupa por estado, mês e produto, somar qtd e ranking
query_top3_exp_mensal = """
WITH agregados AS (
    SELECT SG_UF_NCM AS UF,
           CO_MES,
           CO_NCM,
           SUM(QT_ESTAT) AS QUANTIDADE_TOTAL
    FROM exportacao
    WHERE CO_ANO = 2021
    GROUP BY UF, CO_MES, CO_NCM
),
ranqueados AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY UF, CO_MES ORDER BY QUANTIDADE_TOTAL DESC) AS posicao
    FROM agregados
)
SELECT UF, CO_MES, CO_NCM, QUANTIDADE_TOTAL, posicao
FROM ranqueados
WHERE posicao <= 3
ORDER BY UF, CO_MES, posicao;
"""

top3_exp_mensal = pd.read_sql(query_top3_exp_mensal, conn)
display(top3_exp_mensal)

,UF,CO_MES,CO_NCM,QUANTIDADE_TOTAL,posicao
0,AC,1,10059010,917740,1
1,AC,1,8012100,843300,2
2,AC,1,25171000,167400,3
3,AC,2,8012100,1130980,1
4,AC,2,10059010,452620,2
...,...,...,...,...,...
967,TO,11,2023000,4807838,2
968,TO,11,7133590,445471,3
969,TO,12,10059010,104704574,1
970,TO,12,2023000,6286107,2


In [48]:
# copia do df com os Top 3 produtos por estado
dados_plot = top3_exp_estado.copy()

# verificaçao string
dados_plot["CO_NCM"] = dados_plot["CO_NCM"].astype(str)

# verificaçao int
dados_plot["posicao"] = dados_plot["posicao"].astype(int)

# ordenar o df
dados_plot = dados_plot.sort_values(
    ["UF", "posicao"],
    ascending=[True, True]
).reset_index(drop=True)

# UF'S
ufs = sorted(dados_plot["UF"].unique().tolist())

# Counta quantas UFs para usar no grafico
n_ufs = len(ufs)


In [49]:
#visualizacao top3 mês-estado

# grade
cols = 6  # 6 por linha
rows = math.ceil(n_ufs / cols)  # quantas linhas necessárias
height = max(450, rows * 260)  # altura
width = 1400  # largura

# grafico
fig_grid = px.bar(
    dados_plot,
    x="CO_NCM",                 # eixo X = NCM
    y="QUANTIDADE_TOTAL",       # eixo Y = quantidade exportada
    color="CO_NCM",
    text="QUANTIDADE_TOTAL",    # rótulo
    facet_col="UF",             # gráficos por UF
    facet_col_wrap=cols,
    category_orders={"UF": ufs}, # ordem dos estados
    title="Top 3 produtos mais exportados por UF (2020–2021) — pequenos múltiplos",
    labels={
        "CO_NCM": "Código NCM",
        "QUANTIDADE_TOTAL": "Quantidade total"
    }
)


# ajuste UF=AC em para AC
fig_grid.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))


fig_grid.update_traces(
    texttemplate="%{text:.3s}",  # valores abreviados (1.23M, 4.56k)
    textposition="outside",      # rotulo de dados
    hovertemplate=(
        "<b>UF: %{facet_col}</b><br>"  # destacar UF
        "NCM: %{x}<br>"                # código NCm
        "Qtd: %{y:,}<extra></extra>"   # quantidade formatada com separador de milhar
    )
)

# Criando a própria escala no eixo Y, pq existem muitas discrepâncias entre a exportação nos estados
fig_grid.update_yaxes(matches=None)

fig_grid.update_layout(
    showlegend=False,          # sem legenda
    uniformtext_minsize=8,     # tamanho text
    uniformtext_mode="hide",   # esconde textos que não couberem
    margin=dict(l=30, r=20, t=70, b=40),  # margens
    height=height,             # altura dinâmica
    width=width                # largura
)

fig_grid.show()
